# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류
- 이미지 binary 분류 과제
- 담당: 이녕민M

## Import Libraries

In [1]:
import os, torch, copy, cv2, sys, random, math
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import gc
from torchvision import models
import sklearn.metrics
import seaborn as sn
import matplotlib as plt

## Set Arguments & hyperparameters

In [2]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [5]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= '/USER/Taeyun/'
NUM_CLS = 1

EPOCHS = 30
BATCH_SIZE = 128
LEARNING_RATE = 0.0005
EARLY_STOPPING_PATIENCE = 3
INPUT_SHAPE = 384

os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataloader

#### Train & Validation Set loader

In [7]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        
        # Loading dataset
        self.db= self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db.loc[:int(len(self.db)*0.7),:]
        elif self.mode == 'val':
            self.db = self.db.loc[int(len(self.db)*0.7):,:]
            self.db.reset_index(drop=True,inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([
                                             transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        new_db = pd.read_csv(os.path.join(self.data_dir,'new3.csv'))
        
        return pd.concat([new_db,db],axis=0,ignore_index=True)

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        flag = int(data['file_name'].split('.')[0])>=646
        # Loading image
        if not flag:
            cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        else:
            cvimg = cv2.imread(os.path.join(self.data_dir,'newimg3',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['COVID']


## Model

In [8]:
model = models.AlexNet(num_classes=1)

In [9]:
model.features = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2)),
    nn.BatchNorm2d(64),
    nn.ReLU(inplace=True),
    nn.Dropout2d(0.3),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
    
    nn.Conv2d(64, 192, kernel_size=5, stride=1, padding=2),
    nn.BatchNorm2d(192),
    nn.ReLU(),
    nn.Dropout2d(0.3),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
    
    nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(384),
    nn.ReLU(),
    nn.Dropout2d(0.3),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
    
    nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.Dropout2d(0.3),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
    
    nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [10]:
model.classifier[1] = nn.Sequential(
    nn.Linear(in_features=9216,out_features=4096,bias=True),
    nn.BatchNorm1d(4096)
)
model.classifier[4] =nn.Sequential(
    nn.Linear(in_features=4096,out_features=4096,bias=True),
    nn.BatchNorm1d(4096)
)

In [11]:
model.cuda()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Dropout2d(p=0.3, inplace=False)
    (4): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (6): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): Dropout2d(p=0.3, inplace=False)
    (9): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): Dropout2d(p=0.3, inplace=False)
    (14): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Conv2d(384, 256, 

## Utils
### EarlyStopper

In [12]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)

### Trainer

In [13]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []
        for batch_index, (img, label) in enumerate(dataloader):
            
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = torch.sigmoid(self.model(img)).view(-1)
            
            loss = self.loss_fn(pred, label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend((pred>0.5).int().cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = torch.sigmoid(self.model(img)).view(-1)
            
            loss = self.loss_fn(pred, label)
            val_total_loss += loss.item()
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend((pred>=0.5).int().cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(sklearn.metrics.confusion_matrix(target_lst,pred_lst))
        print(msg)
        return self.validation_score

### Metrics

In [14]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [15]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Loading train dataset..
Loading val dataset..
Train set samples: 10503 Val set samples: 4502


#### Load model and other utils

In [16]:
# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn


# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

### epoch 단위 학습 진행

In [17]:
val_acc=0
for epoch_index in tqdm(range(EPOCHS)):
    trainer.train_epoch(train_dataloader, epoch_index)
    value = trainer.validate_epoch(validation_dataloader, epoch_index)
    
    
    #가장 높은 ACC모델 저장
    if value>=val_acc:
        val_acc = value
        check_point = {
            "model" : model.state_dict(),
            "optimizer" : optimizer.state_dict(),
            "scheduler" : scheduler.state_dict()            
        }
        torch.save(check_point,'best_acc.pt')
    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, 'best_loss.pt')


  0% 0/30 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.7053358271354582, Acc: 0.5304198800342759, F1-Macro: 0.5262189686424702
[[2270    0]
 [2232    0]]
Epoch 0, Val loss: 0.8735940388270786, Acc: 0.504220346512661, F1-Macro: 0.33520378027170705


  3% 1/30 [19:32<9:26:39, 1172.39s/it]

Epoch 1, Train loss: 0.6557978994962645, Acc: 0.6283918880319909, F1-Macro: 0.6283383993308331
[[2270    0]
 [2232    0]]
Epoch 1, Val loss: 1.2722160271235874, Acc: 0.504220346512661, F1-Macro: 0.33520378027170705


  7% 2/30 [38:36<9:03:12, 1164.02s/it]

...

Epoch 14, Train loss: 0.05112589532282294, Acc: 0.9840997810149481, F1-Macro: 0.9840997804383989
[[2268    2]
 [  78 2154]]
Epoch 14, Val loss: 0.046665436189089506, Acc: 0.9822301199466904, F1-Macro: 0.9822187184522464
Validation loss decreased 0.048444523023707525 -> 0.046665436189089506


 50% 15/30 [8:37:30<7:40:47, 1843.17s/it]

Epoch 15, Train loss: 0.05594614912505921, Acc: 0.9797200799771494, F1-Macro: 0.9797200329140549
[[2268    2]
 [  63 2169]]
Epoch 15, Val loss: 0.03946034676794495, Acc: 0.9855619724566859, F1-Macro: 0.9855549872809548
Validation loss decreased 0.046665436189089506 -> 0.03946034676794495


 53% 16/30 [8:57:20<6:24:22, 1647.30s/it]libpng error: Read Error
 53% 16/30 [9:02:51<7:55:00, 2035.72s/it]


OSError: Fail to read 651.png

## Inference
### 모델 로드

In [18]:
LOSS_MODEL_PATH = 'best_loss.pt'
ACC_MODEL_PATH = 'best_acc.pt'

### Load dataset

In [19]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [20]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


### 추론 진행& 결과저장

In [31]:
model.load_state_dict(torch.load(LOSS_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = torch.sigmoid(model(img)).view(-1)
        file_lst.extend(list(file_num))
        pred_lst.extend((pred>=0.5).int().tolist())
       # prob_lst.extend(pred[:, 1].tolist())

1it [00:07,  7.31s/it]


In [32]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('loss_prediction.csv', index=False)

In [33]:
model.load_state_dict(torch.load(ACC_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = torch.sigmoid(model(img)).view(-1)
        file_lst.extend(list(file_num))
        pred_lst.extend((pred>=0.5).int().tolist())
       # prob_lst.extend(pred[:, 1].tolist())

1it [00:07,  7.17s/it]


In [34]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('acc_prediction.csv', index=False)